#### 우선 먼저 twitter API부터 발급 받고 시작
- twitter 데이터를 스크래핑할 수 있는 다양한 패키지들이 많으므로 끌리는 것을 사용하면 될 듯 하다.

In [1]:
import twitter     # pip install twitter

In [2]:
twitter_consumer_key = "-"
twitter_consumer_secret = "-"  
twitter_access_token = "-"
twitter_access_secret = "-"

In [3]:
twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                          consumer_secret=twitter_consumer_secret, 
                          access_token_key=twitter_access_token, 
                          access_token_secret=twitter_access_secret)

In [ ]:
# API가 잘 연결됐는지 확인 : 내 계정에 대한 정보 출력
print(twitter_api.VerifyCredentials())

#### GetUserTimeline() :
https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

In [8]:
# 그래서 일단 기본 세팅은 이렇게
account = "@moonriver365"
statuses = twitter_api.GetUserTimeline(screen_name = account, 
                                       count = 5, include_rts = False, exclude_replies = True, trim_user = True)

In [35]:
print(statuses)   # 하지만 이 방법을 쓰면 조금 긴 text의 끝이 ...로 찍히는 문제점이 발생
print("statuses[0].id : ", statuses[0].id)
print("statuses[0].text : \n", statuses[0].text)
print("statuses[0].created_at : ", statuses[0].created_at)

[Status(ID=1315575174964469760, ScreenName=None, Created=Mon Oct 12 08:48:58 +0000 2020, Text='이번의 방역완화 조치는 \n우리가 코로나를 방역의 통제 속에 \n둘 수 있다는 자신감에 근거한 것입니다.\n\n전 세계적인 코로나 위기 속에서 \n우리는 방역과 경제 모두에서 \n강한 면모를 보여주며 \n‘위기에 강한… https://t.co/BydeIT6vdU'), Status(ID=1314725774545494016, ScreenName=None, Created=Sat Oct 10 00:33:45 +0000 2020, Text='https://t.co/RZx3aI1PSN')]
statuses[0].id :  1315575174964469760
statuses[0].text : 
 이번의 방역완화 조치는 
우리가 코로나를 방역의 통제 속에 
둘 수 있다는 자신감에 근거한 것입니다.

전 세계적인 코로나 위기 속에서 
우리는 방역과 경제 모두에서 
강한 면모를 보여주며 
‘위기에 강한… https://t.co/BydeIT6vdU
statuses[0].created_at :  Mon Oct 12 08:48:58 +0000 2020


#### 잘리는 text 없이  API를 호출할 때 ID와 TOKEN 입력하는 란에 tweet_mode='extended' 추가
- 주의 사항 : tweet_mode='extended'를 추가해서 스크래핑 했을 때는 text가 아닌 full_text라고 적어야 text가 추출된다.

In [30]:
twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                          consumer_secret=twitter_consumer_secret, 
                          access_token_key=twitter_access_token, 
                          access_token_secret=twitter_access_secret, 
                         tweet_mode='extended' )

twitter_api.GetUserTimeline(screen_name = account, 
                                   count = 5, include_rts = False, exclude_replies = True, trim_user = True)[0].full_text

'이번의 방역완화 조치는 \n우리가 코로나를 방역의 통제 속에 \n둘 수 있다는 자신감에 근거한 것입니다.\n\n전 세계적인 코로나 위기 속에서 \n우리는 방역과 경제 모두에서 \n강한 면모를 보여주며 \n‘위기에 강한 나라, 대한민국’으로 \n국가적 위상이 높아지고 있습니다. \n모두가 국민 덕분입니다. https://t.co/HNwRIjijcv'

#### 한 번에 추출할 수 있는 트위터의 갯수가 200개였으므로 
#### since_id와 max_id를 사용해서 모든 트위터가 추출될 수 있게끔 반복문을 돌리려고 함
#### 참고 : https://gist.github.com/freimanas/39f3ad9a5f0249c0dc64

In [ ]:
import pandas as pd

In [ ]:
def get_all_tweets(screen_name):
    global df  
    
    twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                      consumer_secret=twitter_consumer_secret, 
                      access_token_key=twitter_access_token, 
                      access_token_secret=twitter_access_secret, 
                     tweet_mode='extended')     # full_text가 안 보이는 부분은 여기 수정

    alltweets = []

    new_tweets = twitter_api.GetUserTimeline(screen_name = screen_name, count = 1,
                                            include_rts = False, exclude_replies = True, trim_user = True)  # 이런 옵션값은 재량으로 선택하면 됨

    alltweets.extend(new_tweets)

    oldest = alltweets[-1].id - 1

    while len(new_tweets) > 0:
            print("getting tweets before %s" % (oldest))

            new_tweets = twitter_api.GetUserTimeline(screen_name = screen_name, count = 200, 
                                                     max_id = oldest,
                                                    include_rts = False, exclude_replies = True, trim_user = True)  # 이런 옵션값은 재량으로 선택하면 됨

            alltweets.extend(new_tweets)

            oldest = alltweets[-1].id - 1

            print("...%s tweets downloaded so far" % (len(alltweets)))

    outtweets = []
    for tweet in alltweets:
        outtweets.append([tweet.id_str, tweet.created_at, tweet.full_text])

    df = pd.DataFrame(outtweets)
    return df

In [ ]:
# 여러 옵션값으로 돌려본 결과 해당 계정이 보낸 모든 멘션을 스크래핑 하고 싶다면 exclude_replies = False를 선택해야 한다.
# A라는 계정이 B라는 트위터를 올리고 B라는 트위터에 답 멘션으로 B'라는 트위터를 남겼을 때 
# exclude_replies = True를 선택하면 B'의 내용은 스크래핑 되지 않았다.
# A가 다른 사람한테 보낸 답 멘션을 거르고 싶다면 우선 False 값으로 추출한 후
# text의 시작이 @로 되어있는 경우를 제외하면 된다.
# 본인 트위터에 답 멘션을 단 경우에는 @가 붙어있지 않았다.

### 트위터를 올린 날짜와 내용만 수집하고 싶다면 getUserTimeline()을 사용하는 것이 편한 듯 하다.
#### 하지만 getUserTimeline()을 이용해서는 media에 대한 정보를 수집할 수가 없었기 때문에 다른 방법을 찾아보고자 함.